<a href="https://colab.research.google.com/github/ricardocarreras/Bootcamp_DataScience_student/blob/master/semanticsearchparalibro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552447 sha256=7930687127d5a1da63372b071078b97e756a3a5a77d218627b84670d99d35432
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [3]:
from annoy import AnnoyIndex
import numpy as np
import pandas as pd
import re

In [4]:
import os

In [12]:
text = """
este es el texto que quiero analizar. EIZASA nace en 1974. Esto es lo que cuentan de carreras. Así se dice o no se dice lo que interesa. Natividad nació en 1934
"""

In [13]:
# Split into a list of sentences
texts = text.split('.')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])

In [14]:
texts

array(['este es el texto que quiero analizar', 'EIZASA nace en 1974',
       'Esto es lo que cuentan de carreras',
       'Así se dice o no se dice lo que interesa',
       'Natividad nació en 1934'], dtype='<U40')

In [15]:
# Split into a list of paragraphs
texts = text.split('\n\n')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])

In [16]:
texts

array(['este es el texto que quiero analizar. EIZASA nace en 1974. Esto es lo que cuentan de carreras. Así se dice o no se dice lo que interesa. Natividad nació en 1934'],
      dtype='<U160')

In [17]:
# Split into a list of sentences
texts = text.split('.')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])

In [ ]:
title = 'libro sobre Miguel Carreras'

texts = np.array([f"{title} {t}" for t in texts])

In [18]:
texts

array(['este es el texto que quiero analizar', 'EIZASA nace en 1974',
       'Esto es lo que cuentan de carreras',
       'Así se dice o no se dice lo que interesa',
       'Natividad nació en 1934'], dtype='<U40')

In [19]:
pip install cohere

In [20]:
import cohere

In [21]:
co = cohere.Client("vL2rXybxOLRZTswO0lSkE6ku1e0Tgc3lXjcio9Du")

In [ ]:
response =co.embed(
    texts=texts.tolist()
).embeddings

In [24]:
embeds = np.array(response)
embeds.shape

(5, 4096)

In [25]:
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')

True

In [26]:
pd.set_option('display.max_colwidth', None)

def search(query):

  # Get the query's embedding
  query_embed = co.embed(texts=[query]).embeddings

  # Retrieve the nearest neighbors
  similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    3,
                                                  include_distances=True)
  # Format the results
  results = pd.DataFrame(data={'texts': texts[similar_item_ids[0]],
                              'distance': similar_item_ids[1]})

  print(texts[similar_item_ids[0]])

  return results

In [27]:
query = "¿cuándo nació Eizasa?"
search(query)

default model on embed will be deprecated in the future, please specify a model in the request.


['EIZASA nace en 1974' 'Natividad nació en 1934'
 'Esto es lo que cuentan de carreras']


,texts,distance
0,EIZASA nace en 1974,0.647988
1,Natividad nació en 1934,0.672088
2,Esto es lo que cuentan de carreras,0.951989


In [8]:
df = pd.read_excel('/content/base_datos_candidatos_finalpreparada031123.xlsx')